In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [5]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [6]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the party?\n\nBecause it wanted to get a pizza the action!',
 'explanation': 'This joke plays on the double meaning of the word "pizza." In this case, "pizza" can refer both to the food item and to the word "piece of" when pronounced quickly. So, the joke is a play on words: The pizza wanted to get a "piece of" the action at the party. It\'s a light-hearted and punny punchline that relies on the similarity in pronunciation between two words.'}

In [7]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party?\n\nBecause it wanted to get a pizza the action!', 'explanation': 'This joke plays on the double meaning of the word "pizza." In this case, "pizza" can refer both to the food item and to the word "piece of" when pronounced quickly. So, the joke is a play on words: The pizza wanted to get a "piece of" the action at the party. It\'s a light-hearted and punny punchline that relies on the similarity in pronunciation between two words.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-28dd-640e-8002-385855625d65'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-24T05:22:15.899229+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-1441-6797-8001-aa161f7d8afb'}}, tasks=(), interrupts=())

In [8]:
# 4 states : start, get joke , getexp, end
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party?\n\nBecause it wanted to get a pizza the action!', 'explanation': 'This joke plays on the double meaning of the word "pizza." In this case, "pizza" can refer both to the food item and to the word "piece of" when pronounced quickly. So, the joke is a play on words: The pizza wanted to get a "piece of" the action at the party. It\'s a light-hearted and punny punchline that relies on the similarity in pronunciation between two words.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-28dd-640e-8002-385855625d65'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-24T05:22:15.899229+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-1441-6797-8001-aa161f7d8afb'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party?\n\n

In [9]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta go to the party?\n\nBecause it heard it was going to be a spaghetti-noodle-doo!',
 'explanation': 'This joke plays on the pun of "spaghetti-noodle-doo" sounding like "spaghetti doo-doo", which is a playful and silly way of saying that the party is going to be fun and enjoyable. The pasta went to the party because it wanted to join in on the festivities and have a great time. It\'s a light-hearted and humorous play on words that brings a smile to the listener\'s face.'}

In [10]:
workflow.get_state(config2)
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta go to the party?\n\nBecause it heard it was going to be a spaghetti-noodle-doo!', 'explanation': 'This joke plays on the pun of "spaghetti-noodle-doo" sounding like "spaghetti doo-doo", which is a playful and silly way of saying that the party is going to be fun and enjoyable. The pasta went to the party because it wanted to join in on the festivities and have a great time. It\'s a light-hearted and humorous play on words that brings a smile to the listener\'s face.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-37fe-617e-8002-c592ba3d2264'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-24T05:22:17.485541+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-2eef-6eb8-8001-aed6b6a516fa'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta 

Time Travel

In [11]:
workflow.get_state(config1)
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party?\n\nBecause it wanted to get a pizza the action!', 'explanation': 'This joke plays on the double meaning of the word "pizza." In this case, "pizza" can refer both to the food item and to the word "piece of" when pronounced quickly. So, the joke is a play on words: The pizza wanted to get a "piece of" the action at the party. It\'s a light-hearted and punny punchline that relies on the similarity in pronunciation between two words.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-28dd-640e-8002-385855625d65'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-24T05:22:15.899229+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-1441-6797-8001-aa161f7d8afb'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party?\n\n

In [14]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0e0888-0296-6f4b-8000-5ea14bb9ef87"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0e0888-0296-6f4b-8000-5ea14bb9ef87'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-24T05:22:11.885846+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-0295-604a-bfff-df860d3a4ac7'}}, tasks=(PregelTask(id='db95354e-9856-fe0b-1073-8e8505d7bbbe', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the party?\n\nBecause it wanted to get a pizza the action!'}),), interrupts=())

In [15]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0e0888-0296-6f4b-8000-5ea14bb9ef87"}})

{'topic': 'pizza',
 'joke': "Why did the slice of pizza go to the party alone?\n\nBecause it didn't want to be topped with anyone else!",
 'explanation': 'This joke is a play on words with a double meaning. In the context of pizza, "toppings" are the ingredients that are added on top of the pizza crust. So, when it says the slice of pizza didn\'t want to be "topped with anyone else," it means the pizza slice didn\'t want any other toppings added on top of it. Additionally, in social situations, going to a party "alone" can mean without a date or partner. So, the joke is clever in combining both meanings to create a funny scenario of a slice of pizza wanting to go to a party solo to avoid being topped with any other toppings.'}

In [16]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the slice of pizza go to the party alone?\n\nBecause it didn't want to be topped with anyone else!", 'explanation': 'This joke is a play on words with a double meaning. In the context of pizza, "toppings" are the ingredients that are added on top of the pizza crust. So, when it says the slice of pizza didn\'t want to be "topped with anyone else," it means the pizza slice didn\'t want any other toppings added on top of it. Additionally, in social situations, going to a party "alone" can mean without a date or partner. So, the joke is clever in combining both meanings to create a funny scenario of a slice of pizza wanting to go to a party solo to avoid being topped with any other toppings.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0889-23ce-630c-8002-6af9b3afe1b0'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-24T05:22:42.212286+00:00', parent_c

Update State

In [17]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0e0888-0296-6f4b-8000-5ea14bb9ef87", "checkpoint_ns": ""}}, {'topic':'burger'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0e0889-7966-6a82-8001-079debea7082'}}

In [18]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'burger'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0889-7966-6a82-8001-079debea7082'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-12-24T05:22:51.187660+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0888-0296-6f4b-8000-5ea14bb9ef87'}}, tasks=(PregelTask(id='b68f04f4-594b-4b17-a595-e867efc53aab', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the slice of pizza go to the party alone?\n\nBecause it didn't want to be topped with anyone else!", 'explanation': 'This joke is a play on words with a double meaning. In the context of pizza, "toppings" are the ingredients that are added on top of the pizza crust. So, when it says the slice of pizza didn\'t want to be 

Fault tolerance

In [1]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [2]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [3]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [4]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))